- synchronize the dimention of the nodes embeddings (dim =4, each) with the input of the server model.
- handle the final round to save automaticlly the encoders.
- handle the transfert of encoders and preprocessors automaticlly to the server result folder.
- match model architecture between trainning and load for prediction.

we need to get the entire dataset, and extract the relative vital_signs and lab_results,then align them.(all need to be aligned due to VFL logic) And distibute them across nodes.

In [ ]:
import functions

In [ ]:
csv_paths = ['../data/node3_chartevents/data.csv','../data/node4_labevents/data.csv','../data/node1_patients/data.csv']
original_dataframe = functions.merge_csvs_on_feature(csv_paths=csv_paths,merge_on='subject_id')

In [ ]:
_ = functions.analyze_dataframe(original_dataframe)

The next step is to test correlation (linear and !linear) between features to apply the specific algorithm to the specific features, but the problem is that features are distributed across nodes within the VFL environment, so to test correlation we need to creat an other system that can handle this step with privacy preserving.(tip used : we gonna based on the HFL correlations tests.)

In [ ]:
target_column = 'anchor_year'
features = ['anchor_year','respiratory_rate','creatinine']
cleaned_dataframe = functions.prepare_clean_dataset(original_dataframe,features=features)

In [ ]:
_= functions.analyze_dataframe(cleaned_dataframe)

In [ ]:
missing_rate = 0.2
pattern = 'MAR'

data_with_missingness , original_values = functions.introduce_missingness(df=cleaned_dataframe,feature1=features[0],feature2=features[1],missing_rate=missing_rate,task="regression",pattern=pattern)
original_values

In [ ]:
_= functions.analyze_dataframe(data_with_missingness)

In [ ]:
functions.get_nan_rows(data_with_missingness,'anchor_year')

In [ ]:
x,y ,z = functions.load_vfl_model('../server/results/dl_regression')
print('='*50)
print(x)
print('='*50)
print(y)
print('='*50)
print(z)


In [ ]:
predictions=functions.predict_dl(data_with_missingness,'dl_r')
predictions

In [ ]:
functions.analyze_feature(predictions,'prediction')